### Summation of primes

The sum of the primes below **10** is $2 + 3 + 5 + 7 = 17$.

Find the sum of all the primes below two million.

---

#### Natural looping algorithm

We can go though all primes up to two million and adding them together. First we define the same primality checker from problem 7, which consists of a looping algorithm over natural numbers:

In [6]:
def is_prime(n):
    # Preliminary properties
    if n == 1:          # 1 is not a prime number
        return False
    elif n < 4:         # 2 and 3 are prime numbers
        return True
    elif n % 2 == 0:    # Prime numbers greater than 2 are odd
        return False    
    elif n % 3 == 0:    # Get out with prime factor 3 to use next prime property
        return False
    
    # Look for smallest prime factor of n
    else:                      # Prime numbers greater than 3 are of the form 6k+-1
        f_max = int(n**0.5)
        f = 5   # This is a 6k - 1 type number
        while f <= f_max:
            if n % f == 0:
                return False
            elif n % (f + 2) == 0:  # This is a 6k + 1 type number
                return False
            f += 6  # Go to next 6k -1 type number
        return True

We can use it to calculate successive prime numbers and add them up together:

In [7]:
max_prime = int(2e6)    # Constrain
n, s = 3, 2             # 2nd prime number and sum value

while n < max_prime:
    # If prime, add it to the sum
    if is_prime(n):
        p = n
        s += p
    # Go to next number
    n += 2

print(f"Prime number below {max_prime}:", p)
print("Sum value:", s)

Prime number below 2000000: 1999993
Sum value: 142913828922


We know that our prime testing algorithm is a good enhancement from other testing algorithms that divide by all natural numbers, or even by all odd numbers (and do not talk about other algorithms that do not stop at $\sqrt{n}$). However, we can see that this last code took quite a little more time compared to other problems. This is mainly because the number of divisions performed increased tremendously when we look for prime numbers up to one million.

For example, if we want to calculate prime numbers up to five million or 10 million we would have to wait for about 21 s and nearly 1 min, respectively (for an ordinary computer). This is pretty much time and we would like to save it, how do we do this?


#### Sieve of Eratosthenes

The sieve of Eratosthenes is an ancient method to compute the prime numbers below a given limit number $n$. It does so by iteratively marking as composite (i.e., not prime) the multiples of each prime, starting with the first prime number, 2. The multiples of a given prime are generated as a sequence of numbers starting from that prime, with constant difference between them that is equal to that prime. This is easy to see with an example.

##### Example

Consider that we want to calculate all prime numbers below 30. We first star listing all numbers up to 30 starting from 2:

2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30

We take the first prime, which is 2, and go though the list on steps of 2 by 2. This set of numbers are multiples of 2 (i.e. composites) so we erase/sieve them from the list:

2 3 ~~4~~ 5 ~~6~~ 7 ~~8~~ 9 ~~10~~ 11 ~~12~~ 13 ~~14~~ 15 ~~16~~ 17 ~~18~~ 19 ~~20~~ 21 ~~22~~ 23 ~~24~~ 25 ~~26~~ 27 ~~28~~ 29 ~~30~~

We move to the next unsieved number, which is 3, and go through the list on steps of 3 by 3. This are the multiples of 3 and we sieve them as well:

2 3 ~~4~~ 5 ~~6~~ 7 ~~8~~ ~~9~~ ~~10~~ 11 ~~12~~ 13 ~~14~~ ~~15~~ ~~16~~ 17 ~~18~~ 19 ~~20~~ ~~21~~ ~~22~~ 23 ~~24~~ 25 ~~26~~ ~~27~~ ~~28~~ ~~29~~ ~~30~~

For now, the numbers that remain unsieved are those not divisible by 2 or 3. The next number would 5 and sieving their multiples gives:

2 3 ~~4~~ 5 ~~6~~ 7 ~~8~~ ~~9~~ ~~10~~ 11 ~~12~~ 13 ~~14~~ ~~15~~ ~~16~~ 17 ~~18~~ 19 ~~20~~ ~~21~~ ~~22~~ 23 ~~24~~ ~~25~~ ~~26~~ ~~27~~ ~~28~~ ~~29~~ ~~30~~

After sieving by 5, the only numbers left unsieved are those not divisible by 2, 3, or 5. 

The next step would be to continue with the next unsieved number, which is 7, and sieve every 7th number starting from 7. However, they are already crossed out, as they are multiples of smaller primes: 
$$14 = 2 \times 7, \ 21 = 3 \times 7, \ 28 = 4 \times 7$$
The only multiple of 7 that would not be crossed out is $7 \times 7$, but is greater than 30. The numbers not crossed out at this point in the list are all the prime numbers below 30:

2 3 5 7 11 13 17 19 23 29

##### Algorithm

To find all the prime numbers less than or equal to a given integer $n$ by Eratosthenes' method:

1. Create a list of consecutive integers from 2 through $n$: $(2, 3, 4, \dots, n)$.
2. Initially, let $p$ equal 2, the smallest prime number.
3. Enumerate the multiples of $p$ by counting in increments of $p$ from $2p$ to $n$, and mark them in the list (these will be $2p, 3p, 4p, \dots$; the $p$ itself should not be marked).
4. Find the smallest number in the list greater than $p$ that is not marked. If there was no such number, stop. Otherwise, let $p$ now equal this new number (which is the next prime), and repeat from step 3.
5. When the algorithm terminates, the numbers remaining not marked in the list are all the primes below $n$.

The main idea here is that every value given to $p$ will be prime, because if it were composite it would be marked as a multiple of some other, smaller prime. Note that some of the numbers may be marked more than once (e.g., 15 will be marked both for 3 and 5).

- As a refinement, it is sufficient to mark the numbers in step 3 starting from $p^2$, as all the smaller multiples of $p$ will have already been marked at that point. This means that the algorithm is allowed to terminate in step 4 when $p^2$ is greater than $n$.

- Another refinement is to initially list odd numbers only, $(3, 5, \dots, n)$, and count in increments of $2p$ in step 3, thus marking only odd multiples of $p$.

Before jumping directly into the code, is advisable to do some index arithmetics. We are going to consider the list of odd numbers up to $n$
$$3, \, 5, \ 7, \ \dots, \ n$$
and we want to index every number starting from 0 in terms of its value. Let's call $i(p)$ the corresponding index of the number $p$ in this list. We can easily find that 
$$i(p) = \frac{p - 3}{2}, \quad p=3, 5, 7, \dots, n$$
as every odd number can be written in the form $2i + 3$. This will allow us to sieve just odd numbers and directly find the index corresponding to the number immediately below $\sqrt{n}$.

Here is the implementation for the Eratosthenes' method applied to our problem $(n=2 \cdot 10^6)$:

In [33]:
def sieve_of_eratosthenes(n: int) -> list:
    # 1. Create a list of odd numbers represented by a boolean array
    length = (n - 1) // 2
    sieve = [False] * length

    # 2. Start algorithm
    p, i = 3, 0
    while p**2 <= n:
        # Find first unsieved value
        if not sieve[i]:
            # Go through multiples of that value and sieve them
            for q in range(p**2, n + 1, 2*p):
                j = (q - 3) // 2
                sieve[j] = True
        # Next odd number
        p += 2
        i += 1

    # 3. Generate prime numbers with the final sieve array
    prime_list = [2] + [2*i + 3 for i, s in enumerate(sieve) if not s]

    return prime_list

prime_list = sieve_of_eratosthenes(n=int(2e6))

print("List of prime numbers:")
print(prime_list[:10] + ['...'] + prime_list[-10:])
print("Sum:", sum(prime_list))


List of prime numbers:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, '...', 1999853, 1999859, 1999867, 1999871, 1999889, 1999891, 1999957, 1999969, 1999979, 1999993]
Sum: 142913828922
